<a href="https://colab.research.google.com/github/mariabakhareva/MDV/blob/main/%D0%9F%D0%A01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задания к [первой практике](https://colab.research.google.com/drive/1AKldHA9Ib8eQk4yn3yAi-Ffs4EtatPsh?usp=sharing).

### Задание 1



Сбор данных из ИСУ.

Выберите какой-либо факультет, соберите данные по обучающимся на нём группам бакалавров на всех курсах (ИСУ - Учебный процесс - Учебные группы и потоки). Сформируйте таблицу с колонками `группа`, `курс`, `номер_ису`, `фио`. Ответьте на вопросы:

1. Одинаковое ли количество групп от курса к курсу?

2. Есть ли в вашей выборке однофамильцы (для простоты учитывайте только полное совпадение фамилии)? Если да, выведите этих людей.

3. Есть ли в вашей выборке ребята без отчества? Если да, выведите этих людей.

4. Найдите курс, на котором меньше всего обучающихся. Выведите значение и номер курса. Если таких курсов несколько, выведите их все.

5. Для каждого курса найдите группу, в которой больше всего обучающихся. Выведите тройки (курс, группа, кол-во обучающихся). Если для какого-либо курса таких групп окажется несколько, выведите их все.

Старайтесь писать код так, чтобы при изменении наименования факультета ничего не ломалось.

In [ ]:
# Это необходимо, чтобы запустить код в Google Colab. Источник: https://nariyoo.com/python-how-to-run-selenium-in-google-colab/
%%capture
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver-autoinstaller

In [ ]:
import sys
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller


sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--verbose')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1920, 1200')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=chrome_options)  # создание объекта драйвера

In [ ]:
# вход в ИСУ
driver.get("https://id.itmo.ru/login")

# подождать, пока подгрузится страница авторизации
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "username"))  # если случится TimeoutException - перезапустите ячейку
)

<selenium.webdriver.remote.webelement.WebElement (session="8273711a33c54d96440ba5331559a7db", element="7CAE151D604D03546AC2F169AACD7B64_element_13")>

In [ ]:
# информация для входа в ИСУ
username = ''
password = ''

# ввод данных для входа
username_field = driver.find_element(By.NAME, "username")
password_field = driver.find_element(By.NAME, "password")

username_field.send_keys(username)
password_field.send_keys(password)
password_field.send_keys(Keys.RETURN)

In [ ]:
target_faculty_name = 'факультет инфокоммуникационных технологий'
driver.get('https://isu.ifmo.ru')
# driver.page_source - чтобы увидеть содержимое текущей страницы

# PUT YOUR CODE HERE

### Задание 2

Одни из самых ужасных с точки зрения формата представления данные - это данные на сайтах государственных организаций (поликлиники, школы и т.п.). Потренируемся работать с подобными сайтами. Соберем данные за муниципальный этап Всероссийской олимпиады школьников [2022/2023 года по Томской области](http://rcro.tomsk.ru/munitsipal-ny-j-e-tap-vsosh-2022-23-arhiv/).

БОльшая часть кода уже написана за вас, однако вам необходимо дополнить функционал по непосредственному извлечению pandas-датафрейма из файлов различного формата (`pdf`, `xlsx`, `docx`), а также посчитать статистические показатели по собранным данным.

Изучите данные на сайте - таблицы имеют схожую, хоть и слегка различающуюся, структуру. Приведем извлеченную из каждого файла таблицу к датафрейму с колонками `предмет`, `класс`, `фио`, `баллы`. Если что-либо отсутствует - заполним пустой строкой, если речь идёт не о баллах, иначе нулем. ФИО можно расматривать как склейку через пробел полей "Фамилия", "Имя" и "Отчество" в случае наличия последнего.

Обратите внимание, что вы можете менять любую часть кода, добавлять свои функции, а также произвести парсинг полностью вручную (хотя это нежелательно).

In [ ]:
!pip install python-docx xls2xlsx img2table[easyocr]

In [ ]:
from img2table.document import PDF
from img2table.ocr import EasyOCR
from xls2xlsx import XLS2XLSX
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from docx import Document


easyocr = EasyOCR(lang=["ru"], kw={"gpu": False})  # можно использовать любой другой OCR-движок

In [ ]:
def get_file_extension(filename):
    ext = filename.split('.')[-1]
    if ext not in ['xls', 'xlsx', 'pdf', 'docx']:
        raise ValueError
    return ext


def download_file(url, destination):
    response = requests.get(url)
    if response.status_code == 200:
        with open(destination, 'wb') as f:
            f.write(response.content)
        print(f"File downloaded successfully to '{destination}'")
    else:
        print(f"Failed to download file from '{url}'")


def convert_xls_to_xlsx(filename):  # конвертация xlsx-файлов устаревшего образца
    x2x = XLS2XLSX(filename)
    x2x.to_xlsx(filename + "x")
    return filename


def extract_data(subject, grade, filename, ext, verbose=True):
    if ext == 'xlx':
        ext = 'xlsx'
        filename = convert_xls_to_xlsx(filename)

    if ext == 'xlsx':
        df = parse_xlsx(filename)
    elif ext == 'pdf':
        df = parse_pdf(filename)
    elif ext == 'docx':
        df = parse_docx(filename)

    # PUT YOUR CODE HERE - приведение разнородных данных к одному виду

    if verbose:
        print()
        display(df)
        print()

    return df[['предмет', 'класс', 'фио', 'баллы']]


def parse_xlsx(filename):
    # PUT YOUR CODE HERE
    return


def parse_docx(filename):
    # PUT YOUR CODE HERE
    return


def parse_pdf(filename):
    # PUT YOUR CODE HERE
    return

Для простоты будем работать только со следующим списком предметов: экономика, английский язык, математика, биология, информатика и ИКТ. Следующий код менять не нужно.

In [ ]:
target_subjects = ['Экономика', 'Английский язык', 'Математика', 'Биология', 'Информатика и ИКТ']  # рассмотрим только эти предметы

all_dfs = []
response = requests.get('http://rcro.tomsk.ru/munitsipal-ny-j-e-tap-vsosh-2022-23-arhiv/')  # результаты муниципального этапа за 2022-2023 год
main_table = BeautifulSoup(response.text).find('table').findAll('table')[2]  # основная таблица (см. сайт)

for row in main_table.findAll('tr')[1:]:  # первая строка - шапка таблицы
    cells = row.findAll('td')

    subject = cells[0].text.strip()  # название предмета из таблицы
    if subject not in target_subjects:
        continue

    a_hrefs = cells[-1].findAll('a')
    for a_href in a_hrefs:  # ссылки на файлы
        grade = re.search(r'\d+', a_href.text).group(0)
        href = a_href.get('href')

        ext = get_file_extension(href)
        filename = f"{subject}_{grade}.{ext}"
        download_file(href, filename)  # скачивание файла

        extracted_df = extract_data(subject, grade, filename, ext, verbose=True)  # достать данные из файла
        all_dfs.append(extracted_df)

overall_df = pd.concat(all_dfs, ignore_index=True)
overall_df.loc[overall_df['баллы'] == '', 'баллы'] = 0  # фикс для NaN-ов в случае баллов
overall_df['баллы'] = overall_df['баллы'].str.replace(',', '.').astype(float)  # конвертация баллов в числовой формат

Должно получиться что-то похожее:

In [ ]:
overall_df

,предмет,класс,фио,баллы
0,Экономика,9,Зюбин Владислав,84
1,Экономика,9,Кириллов Иван,74
2,Экономика,9,Телицин Артемий,74
3,Экономика,9,Шарабарина Елизавета,72
4,Экономика,9,Кочетов Дмитрий,67
...,...,...,...,...
2051,Информатика и ИКТ,11,Писаров Александр Михайлович,81
2052,Информатика и ИКТ,11,Фрейдман Павел Александрович,80
2053,Информатика и ИКТ,11,Николаева Анастасия Александровна,70
2054,Информатика и ИКТ,11,Матвеева Юлия Алексеевна,9


Постройте сводную таблицу - по каждому предмету для каждого класса посчитайте минимальный балл, средний балл, медианный балл, а также максимальный балл.

Посмотрите на результаты по информатике. О чём могут свидетельствовать подобные цифры?

In [ ]:
overall_df['баллы'] = overall_df['баллы'].apply(lambda val: float(str(val).replace(',', '.')))  # конвертация баллов в числовой формат

# PUT YOUR CODE HERE